In [1]:
from datasets import load_dataset
from datasets import Dataset
import matplotlib.pyplot as plt

/home/vincenzo/miniconda3/envs/flaresense-v2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset('i4ds/radio-sunburst-ecallisto')

Generating test split: 100%|██████████| 9010/9010 [00:00<00:00, 20646.17 examples/s]
